In [ ]:
%matplotlib inline
from matplotlib import pylab

from skimage.io import imread
from skimage import img_as_float

import pandas as pd

from sklearn.cluster import KMeans

import math

# 1
Загрузите картинку parrots.jpg. Преобразуйте изображение, приведя все значения в интервал от 0 до 1. Для этого можно воспользоваться функцией img_as_float из модуля skimage. Обратите внимание на этот шаг, так как при работе с исходным изображением вы получите некорректный результат.

In [ ]:
image=imread("parrots.jpg")

In [ ]:
imageN=img_as_float(image)

# 2
Создайте матрицу объекты-признаки: характеризуйте каждый пиксель тремя координатами - значениями интенсивности в пространстве RGB.

In [ ]:
arr=[]
for i in imageN:
    for j in i:
        arr.append((j[0],j[1],j[2]))

In [ ]:
data=pd.DataFrame(arr,columns=["R","G","B"])

# 3
Запустите алгоритм K-Means с параметрами init='k-means++' и random_state=241. После выделения кластеров все пиксели, отнесенные в один кластер, попробуйте заполнить двумя способами: медианным и средним цветом по кластеру.

In [ ]:
km=KMeans(init='k-means++',random_state=241)

In [ ]:
clusters=km.fit_predict(data)

In [ ]:
clusters=pd.DataFrame(clusters,columns=["Cluster"])

In [ ]:
dataC=pd.concat([data,clusters],axis=1)

In [ ]:
# Среднее
Mean=dataC.pivot_table(["R","G","B"],columns="Cluster",aggfunc="mean")
Mean

In [ ]:
# Медианная
Median=dataC.pivot_table(["R","G","B"],columns="Cluster",aggfunc="median")
Median

In [ ]:
dataMedian=dataC.copy()
for i in range(8):
    dataMedian.R[dataMedian.Cluster==i]=Median[i][2]
    dataMedian.G[dataMedian.Cluster==i]=Median[i][1]
    dataMedian.B[dataMedian.Cluster==i]=Median[i][0]
    
        
dataMean=dataC.copy()
for i in range(8):
    dataMean.R[dataMean.Cluster==i]=Mean[i][2]
    dataMean.G[dataMean.Cluster==i]=Mean[i][1]
    dataMean.B[dataMean.Cluster==i]=Mean[i][0]

# 4 - косяк
Измерьте качество получившейся сегментации с помощью метрики PSNR. Эту метрику нужно реализовать самостоятельно

In [ ]:
# PSNR
# нужно без столбца clucter
def PSNR(a,b):
    arr=[]
    for i in range(a.shape[0]):
        tmp=[]
        for j in range(a.shape[1]):
            tmp.append((a.iloc[i,j]-b.iloc[i,j])**2)
        arr.append(sum(tmp))
    MSE=sum(arr)/len(arr)
    MAX=1#Максимальное значениесреди всех пикселей
    return 10*math.log10(MAX**2/MSE)

In [ ]:
print("Среднее {0}".format(PSNR(dataC.iloc[:,:-1],dataMean.iloc[:,:-1])))
print("Медианная {0}".format(PSNR(dataC.iloc[:,:-1],dataMedian.iloc[:,:-1])))

# 5
Найдите минимальное количество кластеров, при котором значение PSNR выше 20 (можно рассмотреть не более 20 кластеров, но не забудьте рассмотреть оба способа заполнения пикселей одного кластера). Это число и будет ответом в данной задаче.

In [ ]:
# не правильно  тк метрику не правильно реализовал
for j in range(20,21):
    km=KMeans( n_clusters=j,init='k-means++',random_state=241)
    clusters=km.fit_predict(data)
    clusters=pd.DataFrame(clusters,columns=["Cluster"])
    dataC=pd.concat([data,clusters],axis=1)
    Mean=dataC.pivot_table(["R","G","B"],columns="Cluster",aggfunc="mean")
    Median=dataC.pivot_table(["R","G","B"],columns="Cluster",aggfunc="median")
    
    dataMedian=dataC.copy()
    dataMean=dataC.copy()
    for i in range(j):
        dataMedian.R[dataMedian.Cluster==i]=Median[i][2]
        dataMedian.G[dataMedian.Cluster==i]=Median[i][1]
        dataMedian.B[dataMedian.Cluster==i]=Median[i][0]
        dataMean.R[dataMean.Cluster==i]=Mean[i][2]
        dataMean.G[dataMean.Cluster==i]=Mean[i][1]
        dataMean.B[dataMean.Cluster==i]=Mean[i][0]
        
    print(j)    
    print("Среднее {0}".format(PSNR(dataC.iloc[:,:-1],dataMean.iloc[:,:-1])))
    print("Медианная {0}".format(PSNR(dataC.iloc[:,:-1],dataMedian.iloc[:,:-1])))
    print()

In [ ]:
from skimage.measure import compare_psnr # - использую готову метрику

In [ ]:
compare_psnr(dataC.iloc[:,:-1].as_matrix(),dataMean.iloc[:,:-1].as_matrix())

In [ ]:
for j in range(10,21):
    km=KMeans( n_clusters=j,init='k-means++',random_state=241)
    clusters=km.fit_predict(data)
    clusters=pd.DataFrame(clusters,columns=["Cluster"])
    dataC=pd.concat([data,clusters],axis=1)
    Mean=dataC.pivot_table(["R","G","B"],columns="Cluster",aggfunc="mean")
    Median=dataC.pivot_table(["R","G","B"],columns="Cluster",aggfunc="median")
    
    dataMedian=dataC.copy()
    dataMean=dataC.copy()
    for i in range(j):
        dataMedian.R[dataMedian.Cluster==i]=Median[i][2]
        dataMedian.G[dataMedian.Cluster==i]=Median[i][1]
        dataMedian.B[dataMedian.Cluster==i]=Median[i][0]
        dataMean.R[dataMean.Cluster==i]=Mean[i][2]
        dataMean.G[dataMean.Cluster==i]=Mean[i][1]
        dataMean.B[dataMean.Cluster==i]=Mean[i][0]
        
    print(j)    
    print("Среднее {0}".format(compare_psnr(dataC.iloc[:,:-1].as_matrix(),dataMean.iloc[:,:-1].as_matrix())))
    print("Медианная {0}".format(compare_psnr(dataC.iloc[:,:-1].as_matrix(),dataMedian.iloc[:,:-1].as_matrix())  ))
    print()

In [1]:
with open("a1.txt","w") as f:
    f.write("11")